## Import required libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import os
import warnings
warnings.filterwarnings('ignore')

## Task 1 : Data Preparation

In [2]:
df = pd.read_csv('/home/saif/Desktop/Customer Churn Analysis and Prediction/Telco_Customer_Churn_Dataset  (1).csv')

In [3]:
df.drop('customerID', axis=1, inplace=True)

In [27]:
df.sample(10)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
4086,Female,0,Yes,No,72,Yes,Yes,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),79.20,5401.90,No
1664,Female,0,No,No,46,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,No,Bank transfer (automatic),96.05,4399.50,Yes
5850,Male,1,No,No,20,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,No,Credit card (automatic),71.30,1389.20,Yes
5767,Female,0,Yes,No,65,Yes,No,DSL,No,No,No,Yes,Yes,No,Two year,Yes,Credit card (automatic),59.80,3808.20,No
6740,Female,1,No,No,40,Yes,No,Fiber optic,Yes,Yes,No,No,No,No,Month-to-month,Yes,Credit card (automatic),79.20,3233.85,Yes
790,Female,0,Yes,Yes,60,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),59.85,3590.20,No
1654,Male,0,Yes,Yes,1,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,18.80,18.80,No
1597,Male,0,Yes,No,48,Yes,Yes,Fiber optic,Yes,Yes,Yes,No,Yes,No,Two year,Yes,Bank transfer (automatic),96.90,4473.45,No
836,Male,0,Yes,Yes,62,Yes,No,DSL,Yes,No,Yes,Yes,No,Yes,One year,No,Credit card (automatic),71.40,4487.30,No
4060,Male,0,Yes,Yes,54,Yes,Yes,DSL,No,No,Yes,No,No,Yes,Two year,No,Credit card (automatic),63.35,3409.10,No


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7010 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7010 non-null   object 
 1   SeniorCitizen     7010 non-null   int64  
 2   Partner           7010 non-null   object 
 3   Dependents        7010 non-null   object 
 4   tenure            7010 non-null   int64  
 5   PhoneService      7010 non-null   object 
 6   MultipleLines     7010 non-null   object 
 7   InternetService   7010 non-null   object 
 8   OnlineSecurity    7010 non-null   object 
 9   OnlineBackup      7010 non-null   object 
 10  DeviceProtection  7010 non-null   object 
 11  TechSupport       7010 non-null   object 
 12  StreamingTV       7010 non-null   object 
 13  StreamingMovies   7010 non-null   object 
 14  Contract          7010 non-null   object 
 15  PaperlessBilling  7010 non-null   object 
 16  PaymentMethod     7010 non-null   object 
 17  

* As we can see there are 7043 entries
* there are no missing values

In [6]:
# Convert TotalCharges to float
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [10]:
df.duplicated().sum()

0

* There is no duplicate rows in the dataset

In [9]:
#remove duplicates
df.drop_duplicates(inplace=True)

In [13]:
df.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

* No null values in the dataset


In [12]:
#drop rows with missing values
df.dropna(inplace=True)

In [15]:
df.shape

(7010, 20)

### Encoding Categorical columns

In [35]:
df_encoded = df.copy()

In [36]:
categorical_columns = df_encoded.select_dtypes(include=["object"]).columns
categorical_columns

Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Churn'],
      dtype='object')

In [37]:
categorical_columns = [col for col in categorical_columns if col not in ['Churn', 'gender','MultipleLines','InternetService','Contract','PaymentMethod']]
categorical_columns

['Partner',
 'Dependents',
 'PhoneService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'PaperlessBilling']

In [38]:
# Convert all categorical columns
# "Yes" -> 1, "No" -> 0, "No internet service" -> -1
for col in categorical_columns:
    df_encoded[col] = df_encoded[col].apply(lambda x: 1 if x == "Yes" else (-1 if x == "No internet service" else 0))

In [39]:
# gender -> 1 for males, 0 for females
df_encoded['gender'] = df_encoded['gender'].apply(lambda x: 1 if x == "Male" else 0)
df_encoded.sample(10)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
2751,0,0,1,1,19,0,No phone service,DSL,0,0,0,0,0,0,Month-to-month,1,Mailed check,24.85,434.80,No
3328,0,0,0,0,1,1,No,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.25,70.25,No
5246,0,0,0,0,2,1,No,Fiber optic,0,0,0,0,1,0,Month-to-month,1,Electronic check,79.75,164.50,Yes
2387,1,0,0,0,5,1,Yes,Fiber optic,0,1,1,0,1,1,Month-to-month,0,Electronic check,105.30,550.60,No
6886,0,1,0,0,8,1,Yes,Fiber optic,0,0,1,0,1,1,Month-to-month,0,Electronic check,101.35,780.50,Yes
5614,1,0,1,1,68,1,Yes,Fiber optic,1,1,0,0,0,0,One year,1,Bank transfer (automatic),85.50,5696.60,No
6901,0,0,0,0,30,1,Yes,DSL,0,1,1,0,1,0,One year,1,Credit card (automatic),68.95,2038.70,No
1676,0,0,0,0,4,1,Yes,Fiber optic,1,0,0,0,0,0,Month-to-month,1,Electronic check,80.15,319.85,No
1657,1,0,0,0,35,1,Yes,No,-1,-1,-1,-1,-1,-1,Month-to-month,1,Electronic check,25.60,901.25,No
607,1,0,0,1,51,1,No,DSL,0,0,0,1,0,0,One year,0,Credit card (automatic),47.85,2356.75,No


In [40]:
# MultipleLines -> 1 for yes, 0 for no, -1 for No phone service
df_encoded['MultipleLines'] = df_encoded['MultipleLines'].apply(lambda x: 1 if x == "Yes" else (-1 if x == "No phone service" else 0))
df_encoded.sample(10)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
5443,1,0,0,0,26,1,0,DSL,0,0,0,0,0,0,Month-to-month,0,Electronic check,45.45,1233.15,No
5385,1,0,0,0,53,1,1,Fiber optic,0,0,0,0,1,0,Month-to-month,0,Bank transfer (automatic),85.45,4517.25,Yes
3204,1,0,0,1,40,1,0,No,-1,-1,-1,-1,-1,-1,One year,1,Bank transfer (automatic),20.40,854.90,No
5297,0,0,1,1,54,1,0,No,-1,-1,-1,-1,-1,-1,Two year,1,Bank transfer (automatic),20.35,1092.35,No
3853,1,0,0,0,46,1,1,Fiber optic,0,0,0,0,0,0,Month-to-month,0,Bank transfer (automatic),72.80,3249.40,No
4904,0,0,0,0,33,1,1,Fiber optic,0,1,1,0,1,1,Month-to-month,0,Electronic check,107.55,3645.50,No
6458,1,0,1,1,6,1,0,DSL,1,0,0,0,0,0,Month-to-month,1,Electronic check,49.15,295.65,No
4479,0,0,0,0,59,1,1,No,-1,-1,-1,-1,-1,-1,Two year,0,Electronic check,24.45,1493.10,No
5543,0,1,0,0,14,1,0,Fiber optic,1,1,0,0,0,0,Month-to-month,0,Electronic check,78.85,1043.80,No
6087,1,0,0,1,26,0,-1,DSL,0,1,0,0,1,0,One year,1,Bank transfer (automatic),39.95,1023.75,No


In [43]:
# label encoding to churn column as Yes to 1 and no to 0
label_encoder = LabelEncoder()
df_encoded["Churn"] = label_encoder.fit_transform(df["Churn"])
df_encoded.sample(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
3526,1,1,0,0,5,1,0,DSL,0,0,1,0,0,0,Month-to-month,0,Electronic check,50.35,237.25,1
4978,0,0,0,0,1,1,0,DSL,0,0,0,0,0,0,Month-to-month,1,Electronic check,44.10,44.10,1
3571,0,0,0,0,35,1,0,DSL,0,0,1,0,0,1,Month-to-month,1,Electronic check,62.10,2096.10,0
5272,0,0,0,0,6,1,1,Fiber optic,0,1,0,0,1,0,Month-to-month,1,Electronic check,90.10,521.30,1
6536,1,0,0,0,1,0,-1,DSL,0,0,0,0,0,0,Month-to-month,0,Mailed check,24.40,24.40,0


In [44]:
# Unique categories of InternetService
df_encoded['InternetService'].unique()


array(['DSL', 'Fiber optic', 'No'], dtype=object)

In [45]:
# Encode DSL as 1 and Fiber optic as 2 and No as 0
df_encoded['InternetService'] = df_encoded['InternetService'].apply(lambda x: 1 if x == "DSL" else (2 if x == "Fiber optic" else 0))
df_encoded.sample(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
3127,0,0,0,0,29,0,-1,1,0,1,0,0,0,0,Month-to-month,1,Electronic check,30.60,856.35,1
6761,1,0,0,0,1,1,0,0,-1,-1,-1,-1,-1,-1,Month-to-month,1,Mailed check,19.90,19.90,0
290,1,1,0,0,24,1,1,2,0,0,0,0,0,0,Month-to-month,1,Electronic check,76.10,1712.70,1
5562,0,1,1,0,56,1,1,2,0,0,0,1,0,0,Month-to-month,1,Bank transfer (automatic),76.85,4275.75,0
5237,0,0,0,0,26,1,0,1,0,1,1,0,0,0,Month-to-month,1,Credit card (automatic),54.75,1406.90,0


In [46]:
# Unique categories of Contract
df_encoded['Contract'].unique()


array(['Month-to-month', 'One year', 'Two year'], dtype=object)

In [47]:
# Encoding Contract as 1 for Month-to-month, 12 for One year and 24 for Two year
df_encoded['Contract'] = df_encoded['Contract'].apply(lambda x: 1 if x == "Month-to-month" else (12 if x == "One year" else 24))
df_encoded.sample(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
3280,1,0,1,0,55,1,0,1,1,1,1,0,0,0,24,0,Electronic check,58.60,3068.60,0
5996,1,0,0,0,62,1,0,0,-1,-1,-1,-1,-1,-1,24,0,Bank transfer (automatic),19.85,1253.65,0
2287,1,1,1,0,69,1,1,2,0,1,1,1,1,1,24,1,Bank transfer (automatic),108.40,7318.20,1
2774,1,0,0,0,6,1,0,1,0,0,0,1,0,0,1,0,Mailed check,50.05,288.35,0
6112,0,1,1,1,66,1,1,2,0,1,1,0,1,1,12,1,Electronic check,104.90,6891.45,0


In [48]:
# Unique categories of PaymentMethod
df_encoded['PaymentMethod'].unique()

array(['Electronic check', 'Mailed check', 'Bank transfer (automatic)',
       'Credit card (automatic)'], dtype=object)

In [49]:
# Encoding PaymentMethod as 1 for Electronic check, 2 for Mailed check, 3 for Bank transfer (automatic) and 4 for Credit card (automatic)
df_encoded['PaymentMethod'] = df_encoded['PaymentMethod'].apply(lambda x: 1 if x == "Electronic check" else (2 if x == "Mailed check" else (3 if x == "Bank transfer (automatic)" else 4)))
df_encoded.sample(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
5645,1,0,1,0,64,1,1,1,1,0,0,0,1,0,12,1,4,66.15,4392.50,0
229,0,0,1,1,47,1,1,1,1,1,0,1,1,1,24,1,4,85.30,4107.25,0
1891,0,1,0,0,1,1,0,1,0,0,0,0,0,0,1,0,2,45.10,45.10,1
129,1,1,0,0,71,0,-1,1,1,1,0,0,1,1,12,1,1,56.45,3985.35,0
1676,0,0,0,0,4,1,1,2,1,0,0,0,0,0,1,1,1,80.15,319.85,0


In [50]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7010 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7010 non-null   int64  
 1   SeniorCitizen     7010 non-null   int64  
 2   Partner           7010 non-null   int64  
 3   Dependents        7010 non-null   int64  
 4   tenure            7010 non-null   int64  
 5   PhoneService      7010 non-null   int64  
 6   MultipleLines     7010 non-null   int64  
 7   InternetService   7010 non-null   int64  
 8   OnlineSecurity    7010 non-null   int64  
 9   OnlineBackup      7010 non-null   int64  
 10  DeviceProtection  7010 non-null   int64  
 11  TechSupport       7010 non-null   int64  
 12  StreamingTV       7010 non-null   int64  
 13  StreamingMovies   7010 non-null   int64  
 14  Contract          7010 non-null   int64  
 15  PaperlessBilling  7010 non-null   int64  
 16  PaymentMethod     7010 non-null   int64  
 17  

In [51]:
# Save the encoded dataframe to a CSV file
df_encoded.to_csv('/home/saif/Desktop/Customer Churn Analysis and Prediction/encoded_df.csv', index=False)